In [2]:
import pandas as pd
from google.cloud import storage

## Combining All Year Csvs

In [23]:
# Create a Cloud Storage client to download the data a nd upload the model
storage_client = storage.Client()

# Download the data
public_bucket = storage_client.bucket('charitable990') # csv files saved in cloud bucket
blob = public_bucket.blob('2012.csv')
blob.download_to_filename('2012.csv')
blob = public_bucket.blob('2013.csv')
blob.download_to_filename('2013.csv')
blob = public_bucket.blob('2014.csv')
blob.download_to_filename('2014.csv')
blob = public_bucket.blob('2015.csv')
blob.download_to_filename('2015.csv')
blob = public_bucket.blob('2016.csv')
blob.download_to_filename('2016.csv')
blob = public_bucket.blob('2017.csv')
blob.download_to_filename('2017.csv')

df12 = pd.read_csv('2012.csv')
df13 = pd.read_csv('2013.csv')
df14 = pd.read_csv('2014.csv')
df15 = pd.read_csv('2015.csv')
df16 = pd.read_csv('2016.csv')
df17 = pd.read_csv('2017.csv')

df12.shape, df13.shape, df14.shape, df15.shape, df16.shape, df17.shape
# different year dataframes have slightly different dimensions

((294019, 62),
 (289603, 245),
 (299405, 245),
 (294782, 246),
 (307483, 246),
 (300910, 246))

In [25]:
df = pd.concat([df12, df13, df14, df15, df16, df17]) # can combine dfs with different dimensions using pandas concat
df['year'] = df['tax_pd'].astype('str').str[:4].astype('int') # year column
df

,ein,tax_pd,subseccd,unrelbusinccd,initiationfees,grsrcptspublicuse,grsincmembers,grsincother,totcntrbgfts,totprgmrevnue,...,exceeds1pct509,subtotpub509,pubsupplesub509,samepubsuppsubtot509,grsinc509,unreltxincls511tx509,netincunrelatd509,othrinc509,elf,year
0,204122653,200412,7,N,0,0,0,0,0,92545,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004
1,943197474,200512,3,N,0,0,0,0,0,5672,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005
2,204774976,200604,3,N,0,0,0,0,0,115339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006
3,330227205,200612,3,N,0,0,0,0,4200,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006
4,364418787,200612,3,N,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300905,203700526,201612,3,N,0,0,0,0,0,3953058,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,2016
300906,200088838,201606,3,N,0,0,0,0,0,1359916,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,2016
300907,521559828,201606,3,N,0,0,0,0,0,6085974,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,2016
300908,840616041,201612,3,N,0,0,0,0,0,4550226,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,2016


In [27]:
df.year.value_counts()

2015    298344
2014    295716
2013    291711
2016    290986
2012    287598
2011    249449
2017     44167
2010     25425
2009      2037
2008       720
2007        16
2006        10
2003         7
2005         5
2000         5
2004         2
2001         2
2002         1
1976         1
Name: year, dtype: int64

In [28]:
df.to_csv('all_yrs.csv', index = False)

## Processing All Year DF into ML Format:

In [3]:
df = pd.read_csv('all_yrs.csv')
df.shape

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,116,118,119,120,121,122,123,124,125,126,127,128,130,131,132,133,134,135,136,137,139,142,143,245) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1786202, 247)

Finding categorical columns to convert to dummy columns:

In [17]:
nuni = df.nunique()     # number of unqiue values in each columns

In [34]:
# Find values with btwn 2 & 200 unique values, look up in data dictionary to determine if categorical

cat_cols = ['nonpfrea',     # "reason for non-pf status"
           'elf',           # "e-file indicator"
           'subseccd',      # "sub-section id"
           'elfyr-1',
           'nonpfreayr-1'
           ]      
nuni[(nuni < 200) & (nuni > 2)].sort_values()

elf                  4
nonpfrea            17
year                19
subseccd            25
qualhlthreqmntn     33
txexmptint          56
f8282cnt            76
qualhlthonhnd      120
tax_pd             137
totnooforgscnt     155
dtype: int64

X variables:

values from 2013 and 2012 tax year:

In [52]:
# Merging 2013 tax years with 2012 and dropping duplicates (short period returns)
dfx = df[df['year'] == 2013].merge(df[df['year'] == 2012], on = 'ein', suffixes = [None, 'yr-1']).drop_duplicates('ein', keep = False)

# converting categorical columns to numeric
obj_columns = dfx.select_dtypes(['object']).columns 
dfx[obj_columns] = dfx[obj_columns].apply(lambda x: x.astype('category').cat.codes)

dfx = dfx.drop(columns = ['year', 'tax_pd', 'yearyr-1', 'tax_pdyr-1', 'subseccdyr-1']) # dropping columns for year, tax pd, id yr-1

dfx = pd.get_dummies(data = dfx, columns = cat_cols) # dummy columns for categorical cols
dfx

,ein,unrelbusinccd,initiationfees,grsrcptspublicuse,grsincmembers,grsincother,totcntrbgfts,totprgmrevnue,invstmntinc,txexmptbndsproceeds,...,nonpfreayr-1_5,nonpfreayr-1_6,nonpfreayr-1_7,nonpfreayr-1_8,nonpfreayr-1_9,nonpfreayr-1_11,nonpfreayr-1_12,nonpfreayr-1_13,nonpfreayr-1_14,nonpfreayr-1_15
0,910454080,0,0,0,0,0,935508,120315,7456,19969,...,0,0,0,0,0,0,0,0,0,0
1,111966978,0,0,0,0,0,0,5050020,153784,0,...,0,0,0,0,0,0,0,0,0,0
2,420660491,0,0,0,0,0,2945,11440,489,0,...,0,0,0,0,0,0,0,0,0,0
3,530152390,1,220425,468101,0,0,411058,2668044,29316,0,...,0,0,0,0,0,0,0,0,0,0
4,561547207,0,0,0,0,0,75,735052,783,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271751,362882769,0,0,0,0,0,759009,96420,0,0,...,0,0,1,0,0,0,0,0,0,0
271752,46046569,0,0,0,0,0,474098,57374,41797,0,...,0,0,0,0,1,0,0,0,0,0
271753,364324153,0,0,0,0,0,361570,0,0,0,...,0,0,0,0,0,0,0,0,0,0
271754,591311210,0,0,0,0,0,262048,347891,0,0,...,0,0,1,0,0,0,0,0,0,0


Y variables:

Whether org terminated or liquidated in 2014-2016

In [53]:
dfy = df[df['year'].isin([2014,2015,2016])]   # 3 tax years
dfy = dfy[(dfy['ceaseoperationscd'] == 'Y') | (dfy['sellorexchcd'] == 'Y')] # only rows where term or liquidated

#lists of all eins 
term = list(set(list(dfy[dfy['ceaseoperationscd'] == 'Y']['ein'])))
liq = list(set(list(dfy[dfy['sellorexchcd'] == 'Y']['ein'])))
term_or_liq = list(set(list(dfy[(dfy['ceaseoperationscd'] == 'Y') | (dfy['sellorexchcd'] == 'Y')]['ein'])))

len(term), len(liq), len(term_or_liq)

(1718, 2994, 4514)

Merging Y values into X df:

In [54]:
# Creating binary columns for term, liq, term/liq 
dfx['y_term'] = 0
dfx.loc[dfx['ein'].isin(term), 'y_term'] = 1

dfx['y_liq'] = 0
dfx.loc[dfx['ein'].isin(liq), 'y_liq'] = 1

dfx['y_TL'] = 0
dfx.loc[dfx['ein'].isin(term_or_liq), 'y_TL'] = 1

dfx[['y_term', 'y_liq', 'y_TL']].sum()

y_term    1511
y_liq     2542
y_TL      3880
dtype: int64

In [65]:
dfx.shape

(268509, 549)

In [66]:
# Dropping all rows with current or prior year liquidations
dfx = dfx[(dfx['ceaseoperationscd'] != 1)&(dfx['sellorexchcd'] != 1)& \
          (dfx['ceaseoperationscdyr-1'] != 1)&(dfx['sellorexchcdyr-1'] != 1)] 

# Dropping target columns for x years
dfx = dfx.drop(columns = ['ceaseoperationscd', 'sellorexchcd', 'ceaseoperationscdyr-1', 'sellorexchcdyr-1'],
               axis = 1) 

dfx[['y_term', 'y_liq', 'y_TL']].sum()

y_term    1391
y_liq     2376
y_TL      3608
dtype: int64

In [69]:
3608 * 100/ dfx.shape[0]

1.354420448445307

In [68]:
dfx.shape

(266387, 545)

pre-processed data to csv:

In [71]:
dfx.to_csv('all_yrs_processed.csv', index = False)